01_logistic_regression_binary.ipynb: 1 layer (input → output)

In [1]:
import numpy as np

# ----------------------------
# 1) Synthetic Binary Dataset
# ----------------------------
def generate_binary_data(n_per_class=200, seed=0):
    """
    Two Gaussian blobs in 2D for class 0 and class 1.
    Returns:
      X: shape (2*n_per_class, 2)
      Y: shape (2*n_per_class, 1) with values {0,1}
    """
    np.random.seed(seed)
    N = n_per_class
    cov = [[0.5, 0], [0, 0.5]]

    # Class 0 centered at (-1, -1)
    x0 = np.random.multivariate_normal(mean=[-1, -1], cov=cov, size=N)
    y0 = np.zeros((N, 1))

    # Class 1 centered at (+1, +1)
    x1 = np.random.multivariate_normal(mean=[1, 1], cov=cov, size=N)
    y1 = np.ones((N, 1))

    X = np.vstack([x0, x1])  # (2N, 2)
    Y = np.vstack([y0, y1])  # (2N, 1)

    # Shuffle
    perm = np.random.permutation(2 * N)
    return X[perm], Y[perm]

# ----------------------------
# 2) Model: Logistic Regression
# ----------------------------
class LogisticRegression:
    def __init__(self, in_dim, lr=0.1):
        self.W = np.zeros((in_dim, 1))  # shape (2,1)
        self.b = 0.0
        self.lr = lr

    def sigmoid(self, z):
        return 1.0 / (1.0 + np.exp(-z))

    def forward(self, X):
        """
        X: shape (batch, 2)
        returns: shape (batch, 1), probabilities
        """
        z = X.dot(self.W) + self.b      # (batch,1)
        return self.sigmoid(z)

    def compute_loss_and_grad(self, X, Y):
        """
        Binary cross‐entropy loss and gradients
        X: (batch,2), Y: (batch,1)
        Returns:
          loss: scalar
          dW: shape (2,1)
          db: scalar
        """
        m = X.shape[0]
        # Forward
        A = self.forward(X)  # (batch,1)
        # Clip for numerical stability
        A_clipped = np.clip(A, 1e-8, 1 - 1e-8)
        # Loss
        loss = -np.sum(Y * np.log(A_clipped) + (1 - Y) * np.log(1 - A_clipped)) / m

        # Gradients
        dZ = A - Y                    # (batch,1)
        dW = (X.T.dot(dZ)) / m        # (2,1)
        db = np.sum(dZ) / m           # scalar
        return loss, dW, db

    def update_params(self, dW, db):
        self.W -= self.lr * dW
        self.b -= self.lr * db

    def predict(self, X):
        """
        Return {0,1} predictions
        """
        probs = self.forward(X)
        return (probs > 0.5).astype(int)

# ----------------------------
# 3) Training Loop
# ----------------------------
if __name__ == "__main__":
    # Generate data
    X, Y = generate_binary_data(n_per_class=200, seed=0)
    # Split 80% train, 20% val
    split = int(0.8 * X.shape[0])
    X_train, Y_train = X[:split], Y[:split]
    X_val,   Y_val   = X[split:], Y[split:]

    # Instantiate model
    model = LogisticRegression(in_dim=2, lr=0.1)
    epochs = 200

    for epoch in range(1, epochs + 1):
        loss, dW, db = model.compute_loss_and_grad(X_train, Y_train)
        model.update_params(dW, db)

        if epoch % 50 == 0 or epoch == 1:
            # Compute train & val accuracy
            train_preds = model.predict(X_train)
            val_preds   = model.predict(X_val)
            train_acc = np.mean(train_preds == Y_train)
            val_acc   = np.mean(val_preds   == Y_val)
            print(f"Epoch {epoch:3d} | Loss: {loss:.4f} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

    # Final evaluation
    train_acc = np.mean(model.predict(X_train) == Y_train)
    val_acc   = np.mean(model.predict(X_val)   == Y_val)
    print(f"\nFinal Train Acc: {train_acc:.4f} | Final Val Acc: {val_acc:.4f}")

Epoch   1 | Loss: 0.6931 | Train Acc: 0.9656 | Val Acc: 0.9750
Epoch  50 | Loss: 0.2041 | Train Acc: 0.9625 | Val Acc: 0.9750
Epoch 100 | Loss: 0.1496 | Train Acc: 0.9656 | Val Acc: 0.9750
Epoch 150 | Loss: 0.1277 | Train Acc: 0.9656 | Val Acc: 0.9750
Epoch 200 | Loss: 0.1156 | Train Acc: 0.9656 | Val Acc: 0.9750

Final Train Acc: 0.9656 | Final Val Acc: 0.9750
